### Matrix and Data
##### [Hiroyuki Chihara](https://fiomfd.github.io/) (University of the Ryukyus, Okinawa Island, Japan)

1. Data described by matrices
2. Singular value decomposition and low rank approximation
3. Wavelet decomposition with the Haar wavelet and image processing

### 1. Data described by matrices

#### 1-1. Data matrices

Below is a list of height and weight data for 100 people, which can be considered as a $100\times2$ matrix. In general, when the number of individuals is $N$ and each individual has $p$ types of data, the data list can be expressed as a matrix.

Each individual's data can be plotted as a point on a plane with height on the horizontal axis and weight on the vertical axis. This is called a scatter plot. By using pseudoinverse or singular value decomposition in linear algebra, it is possible to obtain the best line that shows the relationship between height and weight using linear regression (the least squares method).

### 1-2. Grayscale images
A grayscale image is a matrix in which each small square (pixel) has an integer between $0$ and $255$, or a real number between $0$ and $1$ obtained by dividing the integer by $255$. The integer $0$ represents black and the integer $255$ represents white. The number represents the intensity of the color, and the higher the number, the stronger the color. For example, the matrix

$\begin{bmatrix}
0 & 15  & 30  & 45 & 60 & 75
\\ 
90 & 105 & 120  & 135  & 150 & 165
\\ 
180 & 195  & 210  & 225 & 240 & 255
\end{bmatrix}$

becomes the following grayscale image:

### 1-3. RGB images

An RGB image is created by combining three matrices of grayscale image data of the same size, each of which is colored with the three primary colors red, green, and blue. The following shows three $16\times16$ matrices generated by the random numbers created by a programming language, and the RGB image by combining the triplet.


### 1-4. Ggrayscale movies
If multiple grayscale images are displayed in succession, we have a grayscale movie. Here, we will generate 100 randomly generated matrices of and animate them.

### 1-5. RGB movies

### 2. Singular value decomposition and low rank approximation
In general an $m{\times}n$ matrix $A=\begin{bmatrix}\vec{a}_1 & \dotsb & \vec{a}_n \end{bmatrix}$ has a specific nonnegative integer which is called as the rank of  $A$ defined by 

$r
:=
\operatorname{dim}\bigl(
\{
A\vec{x}
=
x_1\vec{a}_1+\dotsb+x_n\vec{a}_n 
\ \vert \ 
\vec{x} \in \mathbb{R}^n
\}
\bigr).$ 

It follows that $1 \leqq r \leqq \min\{m,n\}$ unless all the entries of $A$ are zero.  
It is known that there exist  

$\sigma_1 \geqq \dotsb \geqq \sigma_r>0,
\quad
\vec{u}_1,\dotsc,\vec{u}_r \in \mathbb{R}^m,
\quad
\vec{v}_1,\dotsc,\vec{v}_r \in \mathbb{R}^n$

such that we have 

$A=\displaystyle\sum_{j=1}^r\sigma_j\vec{u}_j\vec{v}_j^T.$

Set 

$A_k:=\displaystyle\sum_{j=1}^k\sigma_j\vec{u}_j\vec{v}_j^T, \quad k=1,\dotsc,r.$

It is known that the rank of $A_k$ is $k$ and closest to $A$ among all the $m{\times}n$ matrices of rank $k$, that is, $A_k$ is the best approximation of $A$ among all the $m{\times}n$ matrices of rank $k$.   

We compute $384\times512\times3$ RGB image of fried noodle at a canteen, City University of Hong Kong: 

### 3. Wavelet decomposition and image processing

### 3-1. The Haar wavelet
The discrete wavelet is a pair of orthonomal two vectors $\vec{u}$ and $\vec{v}$ satisfying some condition, and the most typical example of the discrete wavelets is the Haar wavelet. They are very simple vectors: 

$\vec{u}
=
\dfrac{1}{\sqrt{2}}
\begin{bmatrix}
1
\\
1
\\
0
\\
\vdots
\\
0 
\end{bmatrix}, 
\quad
\vec{v}
=
\dfrac{1}{\sqrt{2}}
\begin{bmatrix}
1
\\
-1
\\
0
\\
\vdots
\\
0 
\end{bmatrix}
\in\mathbb{R}^{N},$

Roughly speaking the discrete wavelet transform by the Haar wavelet is to take averages of neighboring elements. The elements of a vector are transformed  to the average of neighboring two elements, and the elements of a matrix are converted to the average of neighboring 2 by 2 elements. The following examples show the outcomes of repeating the discere wavelet tranform twice: 

$\vec{a}_0
=
\begin{bmatrix}
1 \\ 3 \\ 5 \\ 7 
\end{bmatrix}
\mapsto 
\vec{a}_1
=
\begin{bmatrix}
2 \\ 2 \\ 6 \\ 6 
\end{bmatrix}
\mapsto 
\vec{a}_2
=
\begin{bmatrix}
4 \\ 4 \\ 4 \\ 4 
\end{bmatrix}$

$A_0
=
\begin{bmatrix}
0 & 2 & 4 & 6
\\ 
8 & 10 & 12 & 14 
\\ 
16 & 18 & 20 & 22
\\ 
24 & 26 & 28 & 30
\end{bmatrix}
\mapsto 
A_1
=
\begin{bmatrix}
5 & 5 & 9 & 9
\\ 
5 & 5 & 9 & 9 
\\ 
21 & 21 & 25 & 25
\\ 
21 & 21 & 25 & 25
\end{bmatrix}
\mapsto 
A_2
=
\begin{bmatrix}
15 & 15 & 15 & 15
\\ 
15 & 15 & 15 & 15 
\\ 
15 & 15 & 15 & 15
\\ 
15 & 15 & 15 & 15
\end{bmatrix}.$


The output such as $\vec{a}_\ell$ and $A_\ell$ of $\ell$ times operations is said to be the approximation part of level $\ell$, and the remainder term such as $\vec{a}_0-\vec{a}_\ell$ and $A_0-A_\ell$ is called the detail part of level $\ell$. 

We shall observe the wavelet decomposition of image data using the Haar wavelet.

### 3-2. $8\times8$ grayscale image

### 3-3. $16\times16$ RGB image created by random number

### 3-4. $384\times512$ RGB image
fried noodle at a canteen, City University of Hong Kong

 $384=2^7\times3$, $512=2^9$